In [1]:
import numpy as np
import tensorflow as tf
from keras import *
from keras.layers import *
# from keras.layers import Conv2D
from keras import backend as K
from keras.utils import np_utils 
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import *
from keras.optimizers import SGD
import cv2
import glob
import pydot
import graphviz
import pandas as pd

/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# All-CNN-C model

## Modelled after the model proposed in the paper: Striving for Simplicity: The All Convolutional Net

https://arxiv.org/abs/1412.6806

### This model replaces max pooling layers with convolutional layers with stride 2. Each layer has a 3x3 kernel, effectively adding a regularization factor to the model. 

### The data is from a dataset of flower images, with 10 different classes or types of flowers. We will resize the images to be 32x32x3

In [2]:
data = pd.read_csv("data/flower_images/flower_labels.csv")
data["file"] = "data/flower_images/" + data["file"]

In [3]:
X = np.array(data.iloc(1)[0])

In [4]:
y = np.array(data.iloc(1)[1])

In [5]:
def cross_val_split_set(X,portion,y=None):
    '''
    use:
        X = iris.data
        y = iris.target
        X_train, X_test, y_train, y_test = split_set(X,0.1,y) for 10% used as test set
    '''
    X = np.array(X)
    y = np.array(y)
    size = int(X.shape[0]*portion)
    indexlist = np.arange(X.shape[0])
    testinds = np.random.choice(indexlist, size, replace=False)
    traininds = np.array([x for x in range(X.shape[0]) if x not in testinds])  
    if np.all(y == None):
        return X[traininds],X[testinds]
    else:
        return X[traininds],X[testinds],y[traininds],y[testinds]

In [6]:
X_train, X_test, y_train, y_test = cross_val_split_set(X,0.1,y)

In [7]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

In [8]:
def preprocess(path,length,width):
    img = cv2.imread(path)
    img = np.asfarray(cv2.resize(img,(length, width)))
    return img/255.0

def read_bulk(paths):
    result = []
    for i in paths:
        result.append(preprocess(i,32,32))
    return np.array(result)

In [9]:
result = read_bulk(X_train)

In [29]:
class AllCNN():
    """
    All-CNN-C model introduced in the paper "Striving for Simplicity: The All Convolutional Net
    https://arxiv.org/abs/1412.6806
    Replaces pooling layers with conv layers with stride 2
    uses Global Average Pooling instead of a dense layer as well
    Specifically used with images as input
    Parameters:
        X: numpy array data matrix of image pixel intensities
        y: numpy array of labels, to_categorical changes it to a sparse binary matrix
        weights: name of file that denotes weights to load in
    """
    def __init__(self,X,y,weights="None"):
        
        self.X = X
        self.y = to_categorical(y)
        
        self.weights = weights
        self.model = None
        if self.weights == "None":
            self.initialize()
        else:
            self.model = load_model(weights)
        
    def initialize(self):
        
        K.clear_session()

        n_outputs = self.y.shape[1]
        height = self.X.shape[1]
        width = self.X.shape[2]
                
        inp = Input(shape=(height,width,3))
        conv1 = Conv2D(filters=96,kernel_size=3,border_mode='same',activation="relu")(inp)
        conv2 = Conv2D(filters=96,kernel_size=3,border_mode='same',activation="relu")(conv1)
        conv3 = Conv2D(filters=96,kernel_size=3,border_mode='same',strides=2,activation="relu")(conv2)
        dropout1 = Dropout(0.5)(conv3)
        conv4 = Conv2D(filters=192,kernel_size=3,border_mode='same',activation="relu")(dropout1)
        conv5 = Conv2D(filters=192,kernel_size=3,border_mode='same',activation="relu")(conv4)
        conv6 = Conv2D(filters=192,kernel_size=3,border_mode='same',strides=2,activation="relu")(conv5)
        dropout2 = Dropout(0.5)(conv6)
        conv7 = Conv2D(filters=192,kernel_size=3,border_mode='same',activation="relu")(dropout2)
        conv8 = Conv2D(filters=192,kernel_size=1,border_mode='valid',activation="relu")(conv7)
        conv9 = Conv2D(filters=n_outputs,kernel_size=1,border_mode='valid',activation="relu")(conv8)
        global_averaging = GlobalAveragePooling2D()(conv9)
        softmax = Softmax(n_outputs)(global_averaging)
        model = Model(inputs=inp, outputs=softmax)
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
        print(model.summary())
        self.model = model
        
    def save_picture(self,filename):
        plot_model(self.model, to_file=filename)
        
    def train(self,epochs):

        self.model.fit(self.X, self.y ,validation_split=0.1, epochs=epochs,verbose=1)
        self.model.save('saved_models/allcnn.h5')
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        print('Train Accuracy: %f' % (acc*100))
        
    def predict(self,X):
        
        if len(X.shape) == 3:
            X = X.reshape(1,X.shape[0],X.shape[1],X.shape[2])
        predictions = self.model.predict(X)
        return np.argmax(predictions)

In [30]:
allcnn = AllCNN(result,y_train)

/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=96, kernel_size=3, activation="relu", padding="same")`
/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=96, kernel_size=3, activation="relu", padding="same")`
/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=96, kernel_size=3, strides=2, activation="relu", padding="same")`
/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=192, kernel_size=3, activation="relu", padding="same")`
/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 192)       331968    
__________

In [32]:
allcnn.save_picture("../model_images/AllCNN.png")

In [13]:
allcnn.predict(result[0])

5

In [14]:
y_train[0]

array([0])